# Requirement

In [1]:
## jalankan ini dahulu sebelum running
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# A. Case: Detik

In [2]:
## global
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup #scraping web statis
import time
from tqdm import tqdm #info progress

## selenium, scraping web dinamis
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote #parsing string "spasi" menjadi "%20"

In [3]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://www.detik.com'
response = s.get(url)
print(response.status_code)

200


### A1. Scraping satu artikel

In [4]:
## fungsi melakukan scraping satu artikel
def scrape_detik_satu(url: str) -> pd.DataFrame:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)
        try:
            # Pisahkan jadi hari, tanggal, dan jam
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": "\n\n".join(p.get_text(strip=True) for p in soup.find_all('p')),
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "link": url
    }

    df = pd.DataFrame([hasil])
    return df

In [5]:
scrape_detik_satu("https://health.detik.com/berita-detikhealth/d-7933044/joe-biden-disebut-kena-kanker-prostat-agresif-gegara-vaksin-covid-19-ini-faktanya")

,judul,isi,hari,tanggal,jam,kategori,link
0,Joe Biden Disebut Kena Kanker Prostat Agresif ...,Segera setelah mantan presiden AS Joe Biden me...,Senin,26 Mei 2025,13:15 WIB,detikHealthBerita Detikhealth,https://health.detik.com/berita-detikhealth/d-...


### A2. Crawling URLs dengan memasukkan keyword/query
```https://www.detik.com/search/searchall?query=makan%20bergizi%20gratis&page=5&result_type=relevansi``` -> pakai modulo 20

```https://www.detik.com/search/searchall?query={kata_kunci}&page={halaman}&result_type=relevansi```

In [6]:
url = f"https://www.detik.com/search/searchall?query=makan%20bergizi&page=1&result_type=relevansi"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
time.sleep(1)

In [7]:
## fungsi untuk scraping artikel berdasarkan keyword yang diinputkan
def scrape_detik_byquery(kata_kunci, halaman) -> pd.DataFrame:
    # Parameter input
    keyword = kata_kunci
    max_pages = halaman
    results = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    }

    for page in tqdm(range(1, max_pages + 1)):
        encoded_query = quote(keyword)
        url = f"https://www.detik.com/search/searchall?query={encoded_query}&page={page}&result_type=relevansi"
        try:
            response = requests.get(url, headers=headers, timeout=10)
            soup = BeautifulSoup(response.content, 'html.parser')
            time.sleep(1)

            beritas = soup.find_all('div', class_="media__text")
            for berita in beritas:
                try:
                    a = berita.find('a')
                    d = berita.find('div', class_="media__desc")
                    t = berita.find('div', class_="media__date")
                    k = berita.find('h2', class_="media__subtitle")

                    results.append({
                        "judul": a.text.strip() if a else np.nan,
                        "link": a.get('href') if a else np.nan,
                        "desc": d.text.strip() if d else np.nan,
                        "tanggal": t.text.strip() if t else np.nan,
                        "kategori": k.text.strip() if k else np.nan,
                        "keyword": keyword
                    })
                except Exception as e:
                    print("Skip 1 berita:", e)
        except Exception as e:
            print(f"Skip page {page}:", e)

    df = pd.DataFrame(results)
    print("Selesai. Total berita:", len(df))
    return df

In [8]:
df = scrape_detik_byquery("makan bergizi gratis", 3)
df.info()

100%|██████████| 3/3 [00:08<00:00,  2.82s/it]

Selesai. Total berita: 36
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     36 non-null     object
 1   link      36 non-null     object
 2   desc      30 non-null     object
 3   tanggal   36 non-null     object
 4   kategori  36 non-null     object
 5   keyword   36 non-null     object
dtypes: object(6)
memory usage: 1.8+ KB


In [9]:
df.tail(4)

,judul,link,desc,tanggal,kategori,keyword
32,30.000 Sarjana Siap Kelola Program Makan Bergi...,https://finance.detik.com/berita-ekonomi-bisni...,"Kepala BGN Dadan Hindayana mengatakan, 30.000 ...","Selasa, 08 Apr 2025 17:06 WIB",detikFinance,makan bergizi gratis
33,Pangkas Sana-sini demi Makan Bergizi Gratis,https://news.detik.com/x/detail/spotlight/2025...,Kebutuhan anggaran untuk mengeksekusi program ...,"Selasa, 11 Feb 2025 19:07 WIB",detikNews,makan bergizi gratis
34,"Makan Bergizi Gratis, Pendidikan, dan Pemajuan...",https://news.detik.com/kolom/d-7719399/makan-b...,Makan bergizi gratis meningkatkan kesehatan si...,"Senin, 06 Jan 2025 13:30 WIB",detikNews,makan bergizi gratis
35,Makan Bergizi Gratis (Jangan) Bikin Miris,https://news.detik.com/kolom/d-7719323/makan-b...,Makan Bergizi Gratis merupakan program ciamik ...,"Senin, 06 Jan 2025 12:48 WIB",detikNews,makan bergizi gratis


In [10]:
## looping list keyword

# Daftar keyword yang ingin di-scrape
list_keyword = [
    "makan bergizi gratis",
    "efisiensi anggaran",
    "CPNS 2025",
    "kemiskinan world bank"
]

# Set jumlah halaman yang ingin di-scrape per keyword
halaman = 2

# List untuk menyimpan semua DataFrame
dfs = []

for keyword in list_keyword:
    print(f"Scraping untuk keyword: {keyword}")
    df_keyword = scrape_detik_byquery(keyword, halaman)
    dfs.append(df_keyword)

# Gabungkan semua hasil
df_final = pd.concat(dfs, ignore_index=True)

Scraping untuk keyword: makan bergizi gratis


100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


Selesai. Total berita: 24
Scraping untuk keyword: efisiensi anggaran


100%|██████████| 2/2 [00:05<00:00,  2.62s/it]


Selesai. Total berita: 24
Scraping untuk keyword: CPNS 2025


100%|██████████| 2/2 [00:05<00:00,  2.55s/it]


Selesai. Total berita: 20
Scraping untuk keyword: kemiskinan world bank


100%|██████████| 2/2 [00:05<00:00,  2.54s/it]

Selesai. Total berita: 20


In [11]:
df_final.to_csv("./files/detik_links.csv", index=False, encoding='utf-8-sig')
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     88 non-null     object
 1   link      88 non-null     object
 2   desc      80 non-null     object
 3   tanggal   88 non-null     object
 4   kategori  88 non-null     object
 5   keyword   88 non-null     object
dtypes: object(6)
memory usage: 4.3+ KB


### A3. Scraping artikel dari URLs yang telah diperoleh


In [ ]:
def scrape_detik_dari_csv(path_csv: str) -> pd.DataFrame:
    # Membaca CSV hasil scraping link
    df_links = pd.read_csv(path_csv)

    # Memastikan kolom 'link' ada
    if 'link' not in df_links.columns:
        raise ValueError("CSV tidak mengandung kolom 'link'.")

    hasil_semua = []
    for i,row in tqdm(df_links.iterrows(), total=len(df_links)): #tuple (i,series)
        url = row['link']
        df_artikel = scrape_detik_satu(url) #memanggil dan menjalankan fungsi scrape_detik satu artikel
        keyword = row['keyword']
        if df_artikel is not None:
            df_artikel['keyword'] = keyword
            hasil_semua.append(df_artikel)

    # Gabungkan semua DataFrame
    if hasil_semua:
        df_final = pd.concat(hasil_semua, ignore_index=True)
        df_final.to_csv('./files/detik_semua_artikel_query.csv', index=False, encoding='utf-8-sig')
        print("Selesai menyimpan semua artikel.")
        return df_final
    else:
        print("Tidak ada artikel yang berhasil di-scrape.")
        return pd.DataFrame()

In [13]:
df = scrape_detik_dari_csv("./files/detik_links.csv")
df.head()

100%|██████████| 88/88 [03:02<00:00,  2.07s/it]

Selesai menyimpan semua artikel.


,judul,isi,hari,tanggal,jam,kategori,link,keyword
0,"PCO: Pemerintah Siapkan Full Dana MBG, Kalau A...",Kepala Kantor Komunikasi Kepresidenan (PCO) Ha...,Senin,26 Mei 2025,15:09 WIB,detikNewsBerita,https://news.detik.com/berita/d-7933300/pco-pe...,makan bergizi gratis
1,Menteri PU Minta ke Erick Agar CSR 3 BUMN Kary...,Menteri Pekerjaan Umum (PU) Dody Hanggodo memi...,Senin,26 Mei 2025,14:20 WIB,detikFinanceInfrastruktur,https://finance.detik.com/infrastruktur/d-7933...,makan bergizi gratis
2,Kemenkeu Lapor Realisasi Anggaran Makan Bergiz...,Kementerian Keuangan (Kemenkeu) melaporkan rea...,Jumat,23 Mei 2025,16:39 WIB,detikFinanceBerita Ekonomi Bisnis,https://finance.detik.com/berita-ekonomi-bisni...,makan bergizi gratis
3,Video: BGN Bantah Isu Raffi Ahmad Dapat Proyek...,"Kepala Badan Gizi Nasional (BGN), Dadan Hinday...",1,"636 Views | Kamis, 22 Mei 2025",17:03 WIB,NaN,https://20.detik.com/detikupdate/20250522-2505...,makan bergizi gratis
4,Upaya Pemkab Gianyar Sukseskan Program Makan B...,Pemkab Gianyar menunjukkan komitmennya dalam m...,Kamis,22 Mei 2025,15:35 WIB,detikBaliBerita,https://www.detik.com/bali/berita/d-7926942/up...,makan bergizi gratis


### A4. Detik.com menyediakan sitemap xml

In [14]:
# Ambil sitemap (allowed dari detik.com/robots.txt)
url_sitemap = 'https://www.detik.com/sitemap.xml'
response = requests.get(url_sitemap)
soup = BeautifulSoup(response.content, 'xml')  # parsing sebagai XML

# Ambil semua URL sitemap yang disediakan
sitemap_urls = [loc.text for loc in soup.find_all('loc')]
print("Contoh:", sitemap_urls[0])
print("Total sitemap url:", len(sitemap_urls))

Contoh: https://news.detik.com/suara-pembaca/sitemap_web.xml
Total sitemap url: 512


In [15]:
## cek satu sitemap ada berapa artikel
sitemap_berita = sitemap_urls[0] 
resp = requests.get(sitemap_berita)
soup2 = BeautifulSoup(resp.content, 'xml')
artikel_urls = [loc.text.strip() for loc in soup2.find_all('loc')]
print("Total artikel:", len(artikel_urls))

Total artikel: 100


In [16]:
artikel_urls[0]

'https://news.detik.com/suara-pembaca/d-7928775/sulitnya-hapus-data-registrasi-aplikasi-dana'

In [17]:
berita = []
for url in artikel_urls[:5]:  # batasi dulu misalnya 5
    try:
        hasil = scrape_detik_satu(url)
        berita.append(hasil)
    except Exception as e:
        print("Gagal:", url, e)

df = pd.concat(berita, ignore_index=True)
df

,judul,isi,hari,tanggal,jam,kategori,link
0,Sulitnya Hapus Data Registrasi Aplikasi Dana,Saya ingin medaftar akun Dana (PT Espay Debit ...,Jumat,23 Mei 2025,15:32 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928775...
1,Penawaran investasi PT Rifan Berjangka Tak Ses...,Saya adalah nasabah komoditas berjangka dari P...,Jumat,23 Mei 2025,15:15 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928735...
2,"Dilindungi Asuransi, Dua Bulan Lebih Perbaikan...","Pada Oktober 2024, saya membeli mobil Ioniq 5 ...",Jumat,23 Mei 2025,14:33 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928621...
3,Satu Tahun Menunggu Proses Refund Dana Member ...,Kami merupakan salah satu Mitra Member Loket P...,Jumat,16 Mei 2025,11:41 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7916830...
4,"Diklaim Lebih Kuat dari Kaca dan Acrylic, Sola...",Saya membeli Solartuff dari Impact Pratama Gro...,Selasa,13 Mei 2025,22:57 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7912935...


### A5. Scraping seluruh isi sitemap

In [ ]:
#fungsi ambil url sitemap dari sitemap utama
def get_sitemap_urls(master_sitemap_url: str):
    response = requests.get(master_sitemap_url)
    soup = BeautifulSoup(response.content, 'xml')
    sitemap_urls = [loc.get_text().strip() for loc in soup.find_all('loc')]
    return sitemap_urls

#fungsi ambil url article
def get_article_urls(sitemap_url: str):
    response = requests.get(sitemap_url)
    soup = BeautifulSoup(response.content, 'xml')
    article_urls = [loc.get_text().strip() for loc in soup.find_all('loc')]
    return article_urls

def scrape_detik(url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal utuh
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)
        try:
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": "\n\n".join(p.get_text(strip=True) for p in soup.find_all('p')),
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "link": url
    }

    return hasil

In [23]:
def scrape_all_from_sitemap(master_sitemap_url: str, max_sitemap=None, max_articles=None, output_csv: str="detik_semua_artikel_sitemap.csv"):
    """
    Robot scraping semua artikel dari sitemap utama Detik.

    Args:
        master_sitemap_url (str): URL sitemap utama.
        max_sitemap (int, optional): Batasi jumlah sitemap yang di-scrape. None artinya semua.
        max_articles (int, optional): Batasi jumlah artikel per sitemap. None artinya semua.
        output_csv (str): Nama file output CSV.

    Returns:
        pd.DataFrame: DataFrame berisi hasil scraping artikel.
    """
    all_articles = []

    print("Mengambil daftar sitemap...")
    sitemap_urls = get_sitemap_urls(master_sitemap_url)

    if max_sitemap:
        sitemap_urls = sitemap_urls[:max_sitemap]

    for sitemap in tqdm(sitemap_urls, desc="Sitemap"):
        try:
            article_urls = get_article_urls(sitemap)
            if max_articles:
                article_urls = article_urls[:max_articles]

            for url in tqdm(article_urls, desc="Artikel", leave=False):
                try:
                    article = scrape_detik(url)
                    all_articles.append(article)
                    time.sleep(1)  # Hindari ban
                except Exception as e:
                    print("Gagal scraping artikel:", url, e)

        except Exception as e:
            print("Gagal akses sitemap:", sitemap, e)

    # Simpan ke CSV
    df = pd.DataFrame(all_articles)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"Selesai. Total artikel: {len(df)}. Hasil disimpan di {output_csv}")
    return df

In [24]:
df = scrape_all_from_sitemap('https://www.detik.com/sitemap.xml', max_sitemap=2, max_articles=10)
df.head()

Mengambil daftar sitemap...


Sitemap: 100%|██████████| 2/2 [00:57<00:00, 28.75s/it]

Selesai. Total artikel: 20. Hasil disimpan di detik_semua_artikel_sitemap.csv


,judul,isi,hari,tanggal,jam,kategori,link
0,Sulitnya Hapus Data Registrasi Aplikasi Dana,Saya ingin medaftar akun Dana (PT Espay Debit ...,Jumat,23 Mei 2025,15:32 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928775...
1,Penawaran investasi PT Rifan Berjangka Tak Ses...,Saya adalah nasabah komoditas berjangka dari P...,Jumat,23 Mei 2025,15:15 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928735...
2,"Dilindungi Asuransi, Dua Bulan Lebih Perbaikan...","Pada Oktober 2024, saya membeli mobil Ioniq 5 ...",Jumat,23 Mei 2025,14:33 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928621...
3,Satu Tahun Menunggu Proses Refund Dana Member ...,Kami merupakan salah satu Mitra Member Loket P...,Jumat,16 Mei 2025,11:41 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7916830...
4,"Diklaim Lebih Kuat dari Kaca dan Acrylic, Sola...",Saya membeli Solartuff dari Impact Pratama Gro...,Selasa,13 Mei 2025,22:57 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7912935...


# B. Case: Tempo

In [25]:
## global
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup #scraping web statis
import time
from tqdm import tqdm #info progress

## selenium, scraping web dinamis
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote_plus #parsing string "spasi" menjadi "+"

In [26]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://www.tempo.co/search?q=makan+bergizi+gratis&page=1'
response = s.get(url)
print(response.status_code)

200


### B1. Scraping satu artikel

In [27]:
##Fungsi melakukan scraping data 1 halaman Tempo, masukkan string url
def scrape_tempo(url: str) -> pd.DataFrame: 
    ## Inisiasi dictionary hasil
    hasil = {}

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        ##Judul
        judul = soup.find('h1', class_='text-[26px] font-bold leading-[122%] text-neutral-1200')
        hasil['judul'] = judul.get_text(strip=True) if judul else np.nan

        ## Sub judul
        sub_judul = soup.find('div', class_='font-roboserif leading-[156%] text-neutral-1100')
        hasil['sub_judul'] = sub_judul.get_text(strip=True) if sub_judul else np.nan

        ## Isi berita
        isi_paragraf = []
        isi_berita = soup.find_all('div', id='content-wrapper', class_='max-lg:container xl')

        for i in isi_berita:
            paragraf = i.find_all('p')
            for p in paragraf:
                teks = p.get_text(strip=True)
                if teks:  #menambahkan teks bila ada
                    isi_paragraf.append(teks)
        ringkasan = '\n\n'.join(isi_paragraf)
        hasil['isi'] = ringkasan if ringkasan else np.nan

        ## Tanggal & Jam publikasi
        tanggal_publikasi = soup.find('p', class_='text-neutral-900 text-sm')
        if tanggal_publikasi:
            waktu = tanggal_publikasi.get_text(strip=True)
            if '|' in waktu:
                tanggal, jam = [part.strip() for part in waktu.split('|')]
                hasil['tanggal'] = tanggal
                hasil['jam'] = jam
            else:
                hasil['tanggal'] = waktu
                hasil['jam'] = np.nan
        else:
            hasil['tanggal'] = np.nan
            hasil['jam'] = np.nan

        ## Kategori
        kategori = soup.find('span', class_='text-sm font-medium text-primary-main')
        hasil['kategori'] = kategori.get_text(strip=True) if kategori else np.nan

        ## Link
        hasil['link'] = url 

    except Exception as e:
        print(f"Terjadi kesalahan saat scraping: {e}")
        return None

    ## Kembalikan juga sebagai DataFrame
    df = pd.DataFrame([hasil])
    # print('selesai scraping')
    return df

In [28]:
url = 'https://www.tempo.co/ekonomi/potensi-masalah-dari-rencana-pemerintah-ubah-lapas-jadi-perumahan-1533913'
df_hasil = scrape_tempo(url)
df_hasil

,judul,sub_judul,isi,tanggal,jam,kategori,link
0,Potensi Masalah dari Rencana Pemerintah Ubah L...,Rencana ini berpotensi melanggar hak asasi par...,"TEMPO.CO,Jakarta- Menteri Perumahan dan Kawasa...",24 Mei 2025,21.00 WIB,Bisnis,https://www.tempo.co/ekonomi/potensi-masalah-d...


### B2. Crawling URLs

In [32]:
##Fungsi melakukan scraping URL Tempo, masukkan string keyword dan max halaman
def scrape_tempo_search_selenium(kata_kunci: str, halaman: int) -> pd.DataFrame: 
    """
    Robot crawling url yang diinginkan berdasarkan kata kunci yang user input.

    Args:
        kata_kunci (str): Query yang ingin dimasukkan.
        halaman (int): Batasi jumlah halaman yang di-scrape.

    Returns:
        pd.DataFrame: DataFrame berisi hasil crawling url.
    """
    # Set User-Agent
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
    opts = Options()
    opts.add_argument(f"user-agent={user_agent}")
    opts.add_argument("--headless")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
        
    # Parameter input
    keyword = kata_kunci #contoh: "makan bergizi gratis"
    max_pages = halaman #contoh: 2
    results = []

    # Loop halaman
    for page in tqdm(range(1, max_pages + 1)):
        # print(f"Scraping page {page}...")

        # Format URL pencarian
        encoded_query = quote_plus(keyword)
        url = f"https://www.tempo.co/search?q={encoded_query}&page={page}"
        
        driver.get(url)
        time.sleep(10)

        try:
            container = driver.find_element("css selector", "div.flex.flex-col.divide-y.divide-neutral-500")
            beritas = container.find_elements("css selector", "figure.flex.flex-row.gap-3.py-4.container.lg\\:mx-0.lg\\:px-0")
            for berita in beritas:
                try:
                    a = berita.find_element("tag name", "a")
                    p = berita.find_element("tag name", "p")
                    results.append({
                        "judul": p.text,
                        "link": a.get_attribute("href"),
                        "keyword": keyword
                    })
                except Exception as e:
                    print("Skip 1 berita:", e)

        except Exception as e:
            print("Skip page:", e)

    driver.quit()

    # Simpan ke DataFrame
    df = pd.DataFrame(results)
    print("Selesai. Total berita:", len(df))
    return df

In [33]:
#max pages yang bisa diakses publik hanya 100 pages untuk tiap keyword (pengecekan manual)
scrape_tempo_search_selenium("makan bergizi gratis", 2)

100%|██████████| 2/2 [00:22<00:00, 11.46s/it]


Selesai. Total berita: 20


,judul,link,keyword
0,Sinyal Darurat Penerimaan Pajak,https://www.tempo.co/kolom/rasio-pajak-penerim...,makan bergizi gratis
1,Seknas Fitra: Makan Bergizi Gratis Belum Dongk...,https://www.tempo.co/ekonomi/seknas-fitra-maka...,makan bergizi gratis
2,"Hingga 21 Mei, Anggaran Makan Bergizi Gratis S...",https://www.tempo.co/ekonomi/hingga-21-mei-ang...,makan bergizi gratis
3,Korban Keracunan MBG Lebih Besar dari Klaim Pr...,https://www.tempo.co/politik/korban-keracunan-...,makan bergizi gratis
4,"Sejumlah Titik Kritis dalam Program MBG, Menur...",https://www.tempo.co/politik/sejumlah-titik-kr...,makan bergizi gratis
5,Bos BGN Membantah Raffi Ahmad Kelola 300 Dapur...,https://www.tempo.co/ekonomi/bos-bgn-membantah...,makan bergizi gratis
6,Sekjen Kemendagri Instruksikan Percepatan Laha...,https://www.tempo.co/info-tempo/sekjen-kemenda...,makan bergizi gratis
7,Badan Gizi Nasional jadi Instansi dengan Pagu ...,https://www.tempo.co/ekonomi/badan-gizi-nasion...,makan bergizi gratis
8,BGN Tambah 294 SPPG untuk Makan Bergizi Gratis...,https://www.tempo.co/politik/bgn-tambah-294-sp...,makan bergizi gratis
9,REI: Program 3 Juta Rumah Dukung Efektivitas M...,https://www.tempo.co/ekonomi/rei-program-3-jut...,makan bergizi gratis


In [34]:
## looping list keyword

# Daftar keyword yang ingin di-scrape
list_keyword = [
    "makan bergizi gratis",
    "efisiensi anggaran",
    "CPNS 2025",
    "kemiskinan world bank"
]

# Set jumlah halaman yang ingin di-scrape per keyword
halaman = 2

# List untuk menyimpan semua DataFrame
dfs = []

for keyword in list_keyword:
    print(f"Scraping untuk keyword: {keyword}")
    df_keyword = scrape_tempo_search_selenium(keyword, halaman)
    dfs.append(df_keyword)

# Gabungkan semua hasil
df_final = pd.concat(dfs, ignore_index=True)

Scraping untuk keyword: makan bergizi gratis


100%|██████████| 2/2 [00:32<00:00, 16.12s/it]


Selesai. Total berita: 20
Scraping untuk keyword: efisiensi anggaran


100%|██████████| 2/2 [00:25<00:00, 12.59s/it]


Selesai. Total berita: 20
Scraping untuk keyword: CPNS 2025


100%|██████████| 2/2 [00:28<00:00, 14.02s/it]


Selesai. Total berita: 20
Scraping untuk keyword: kemiskinan world bank


100%|██████████| 2/2 [00:25<00:00, 12.89s/it]


Selesai. Total berita: 14


In [35]:
df_final.to_csv("./files/tempo_links.csv", index=False, encoding='utf-8-sig')
df_final

,judul,link,keyword
0,Sinyal Darurat Penerimaan Pajak,https://www.tempo.co/kolom/rasio-pajak-penerim...,makan bergizi gratis
1,Seknas Fitra: Makan Bergizi Gratis Belum Dongk...,https://www.tempo.co/ekonomi/seknas-fitra-maka...,makan bergizi gratis
2,"Hingga 21 Mei, Anggaran Makan Bergizi Gratis S...",https://www.tempo.co/ekonomi/hingga-21-mei-ang...,makan bergizi gratis
3,Korban Keracunan MBG Lebih Besar dari Klaim Pr...,https://www.tempo.co/politik/korban-keracunan-...,makan bergizi gratis
4,"Sejumlah Titik Kritis dalam Program MBG, Menur...",https://www.tempo.co/politik/sejumlah-titik-kr...,makan bergizi gratis
...,...,...,...
69,"Berdiri Pasca Perang Dunia II, Apa Peran Bank ...",https://www.tempo.co/ekonomi/berdiri-pasca-per...,kemiskinan world bank
70,"76 Tahun Bank Dunia, Targetnya Akhiri Kemiskin...",https://www.tempo.co/ekonomi/76-tahun-bank-dun...,kemiskinan world bank
71,"World Bank: Kemiskinan Terendah di Jakarta, Te...",https://www.tempo.co/ekonomi/world-bank-kemisk...,kemiskinan world bank
72,Bank Dunia: Urbanisasi Naik 1 Persen Turunkan ...,https://www.tempo.co/ekonomi/bank-dunia-urbani...,kemiskinan world bank


### B3. Scraping artikel dalam URLs

In [36]:
##Fungsi melakukan scraping data dari hasil crawling URL Tempo, masukkan csv
def scrape_tempo_dari_csv(path_csv: str) -> pd.DataFrame:
    # Membaca CSV hasil scraping link
    df_links = pd.read_csv(path_csv)

    # Memastikan kolom 'link' ada
    if 'link' not in df_links.columns:
        raise ValueError("CSV tidak mengandung kolom 'link'.")

    hasil_semua = []
    for i, row in tqdm(df_links.iterrows(), total=len(df_links)):
        url = row['link']
        df_artikel = scrape_tempo(url) #memanggil dan menjalankan fungsi scrape_tempo satu artikel
        keyword = row['keyword']
        if df_artikel is not None:
            df_artikel['keyword'] = keyword
            hasil_semua.append(df_artikel)

    # Gabungkan semua DataFrame
    if hasil_semua:
        df_final = pd.concat(hasil_semua, ignore_index=True)
        df_final.to_csv('./files/tempo_semua_artikel.csv', index=False, encoding='utf-8-sig')
        print("Selesai menyimpan semua artikel.")
        return df_final
    else:
        print("Tidak ada artikel yang berhasil di-scrape.")
        return pd.DataFrame()

In [37]:
df = scrape_tempo_dari_csv("./files/tempo_links.csv")
df.head(5)

100%|██████████| 74/74 [02:09<00:00,  1.75s/it]

Selesai menyimpan semua artikel.


,judul,sub_judul,isi,tanggal,jam,kategori,link,keyword
0,Sinyal Darurat Penerimaan Pajak,Pemerintah tidak kunjung memperbaiki rasio per...,JEBLOKNYApenerimaan pajakpada kuartal pertama ...,26 Mei 2025,06.00 WIB,Editorial,https://www.tempo.co/kolom/rasio-pajak-penerim...,makan bergizi gratis
1,Seknas Fitra: Makan Bergizi Gratis Belum Dongk...,Karena memaksakan Makan Bergizi Gratis pemerin...,"TEMPO.CO,Jakarta-Sekretariat Nasional Forum In...",24 Mei 2025,17.24 WIB,Bisnis,https://www.tempo.co/ekonomi/seknas-fitra-maka...,makan bergizi gratis
2,"Hingga 21 Mei, Anggaran Makan Bergizi Gratis S...",Hingga 21 Mei 2025 kementerian keuangan telah ...,"TEMPO.CO,Jakarta- Wakil Menteri Keuangan Suaha...",23 Mei 2025,16.11 WIB,Bisnis,https://www.tempo.co/ekonomi/hingga-21-mei-ang...,makan bergizi gratis
3,Korban Keracunan MBG Lebih Besar dari Klaim Pr...,Tim Cek Fakta Tempo menemukan angka keracunan ...,"TEMPO.CO,Jakarta-PRESIDEN Prabowo Subianto men...",23 Mei 2025,14.10 WIB,Politik,https://www.tempo.co/politik/korban-keracunan-...,makan bergizi gratis
4,"Sejumlah Titik Kritis dalam Program MBG, Menur...",Kepala BPOM mendorong keterlibatan instansinya...,KOMISI IX DPR mendesak Badan Pengawas Obat dan...,23 Mei 2025,10.13 WIB,Politik,https://www.tempo.co/politik/sejumlah-titik-kr...,makan bergizi gratis


## Next: Analisis Sentimen